Deep Learning
====
Assignment 4
----
Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify notMNIST characters.

The goal of this assignment is make the neural network convolutional.


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.



In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run() #tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.024334
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.756649
Minibatch accuracy: 31.2%
Validation accuracy: 50.5%
Minibatch loss at step 100: 1.482760
Minibatch accuracy: 50.0%
Validation accuracy: 71.8%
Minibatch loss at step 150: 1.012287
Minibatch accuracy: 81.2%
Validation accuracy: 74.5%
Minibatch loss at step 200: 0.785661
Minibatch accuracy: 75.0%
Validation accuracy: 77.6%
Minibatch loss at step 250: 0.841277
Minibatch accuracy: 81.2%
Validation accuracy: 76.9%
Minibatch loss at step 300: 0.527110
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.541709
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.403342
Minibatch accuracy: 93.8%
Validation accuracy: 79.7%
Minibatch loss at step 450: 0.673169
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss at step 500: 0.684902
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
M

Problem 1
----
The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # stride 2, kernal size 2
    maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(maxpool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    shape = maxpool.get_shape().as_list()
    reshape = tf.reshape(maxpool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run() #tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.566590
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.740117
Minibatch accuracy: 37.5%
Validation accuracy: 50.9%
Minibatch loss at step 200: 1.039900
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Minibatch loss at step 300: 0.624237
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.407064
Minibatch accuracy: 87.5%
Validation accuracy: 79.7%
Minibatch loss at step 500: 0.564757
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 600: 0.451277
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 700: 0.630809
Minibatch accuracy: 81.2%
Validation accuracy: 83.1%
Minibatch loss at step 800: 0.374371
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 900: 0.281046
Minibatch accuracy: 87.5%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.297648
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%

Problem 2
---

Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.

In [63]:
# LeNet-5: c1->s2->c3->s4->c5->f6->output
batch_size = 16
patch_size = 5
# C1 is a convolutional layer withfeature maps
# Each unit in each feature map is connected to a 5x5 
# neighborhood in the input.
c1_depth = 6 
# C3 is a convolutional layer with 16 feature maps
# Each unit in each feature map is connected to a 5x5 
# neighborhood in the input.
c3_depth = 16
# C5 is a convolutional layer with 120 feature maps
# Each unit in each feature map is connected to a 5x5 
# neighborhood in the input.
c5_depth = 120
# F6 contains 84 units
num_hidden = 84
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  c1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, c1_depth], stddev=0.1))
  c1_biases = tf.Variable(tf.zeros([c1_depth]))
  c3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, c1_depth, c3_depth], stddev=0.1))
  c3_biases = tf.Variable(tf.constant(1.0, shape=[c3_depth]))
  c5_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, c3_depth, c5_depth], stddev=0.1))
  c5_biases = tf.Variable(tf.constant(1.0, shape=[c5_depth]))
  c5_conv_dim = (((((image_size+1)//2)+1)//2)+1)//2
  f6_weights = tf.Variable(tf.truncated_normal(
    [c5_conv_dim * c5_conv_dim * c5_depth, num_hidden], stddev=0.1)) 
  f6_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  out_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  out_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, c1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + c1_biases)
    maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(maxpool, c3_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + c3_biases)
    maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(maxpool, c5_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + c5_biases)
    #maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # do i need subsampling????
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, f6_weights) + f6_biases)
    return tf.matmul(hidden, out_weights) + out_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [64]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run() #tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7.702826
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 200: 0.975347
Minibatch accuracy: 75.0%
Validation accuracy: 77.7%
Minibatch loss at step 400: 0.218958
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Minibatch loss at step 600: 0.405701
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 800: 0.533377
Minibatch accuracy: 81.2%
Validation accuracy: 82.8%
Minibatch loss at step 1000: 0.372664
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 1200: 0.695374
Minibatch accuracy: 81.2%
Validation accuracy: 84.3%
Minibatch loss at step 1400: 1.087246
Minibatch accuracy: 75.0%
Validation accuracy: 84.1%
Minibatch loss at step 1600: 0.251621
Minibatch accuracy: 93.8%
Validation accuracy: 84.9%
Minibatch loss at step 1800: 0.748000
Minibatch accuracy: 75.0%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 0.861457
Minibatch accuracy: 68.8%
Validation accuracy: 

In [94]:
# LeNet-5: c1->s2->c3->s4->c5->f6->output
batch_size = 128 #16
patch_size = 5
# C1 is a convolutional layer withfeature maps
# Each unit in each feature map is connected to a 5x5 
# neighborhood in the input.
c1_depth = 6 
# C3 is a convolutional layer with 16 feature maps
# Each unit in each feature map is connected to a 5x5 
# neighborhood in the input.
c3_depth = 16
# C5 is a convolutional layer with 120 feature maps
# Each unit in each feature map is connected to a 5x5 
# neighborhood in the input.
c5_depth = 120
# F6 contains 84 units
num_hidden = 84
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  c1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, c1_depth], stddev=0.1))
  c1_biases = tf.Variable(tf.zeros([c1_depth]))
  c3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, c1_depth, c3_depth], stddev=0.1))
  c3_biases = tf.Variable(tf.constant(1.0, shape=[c3_depth]))
  c5_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, c3_depth, c5_depth], stddev=0.1))
  c5_biases = tf.Variable(tf.constant(1.0, shape=[c5_depth]))
  c5_conv_dim = (((((image_size+1)//2)+1)//2)+1)//2
  f6_weights = tf.Variable(tf.truncated_normal(
    [c5_conv_dim * c5_conv_dim * c5_depth, num_hidden], stddev=0.01)) #
  f6_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  out_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  out_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, c1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + c1_biases)
    maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(maxpool, c3_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + c3_biases)
    maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(maxpool, c5_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + c5_biases)
    #maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # dropout should be applied between 
    # the final convolutiona and the first fully connected lyaer 
    drop = tf.nn.dropout(hidden, 1.0)
    
    shape = drop.get_shape().as_list()
    reshape = tf.reshape(drop, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, f6_weights) + f6_biases)
    return tf.matmul(hidden, out_weights) + out_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  beta = 1e-3  #L-2 regularization which penalizes large weights
  R = tf.nn.l2_loss(c1_weights) + tf.nn.l2_loss(c3_weights) + tf.nn.l2_loss(c5_weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta * R
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 100, 0.96)
  
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run() #tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 2.745201
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.676829
Minibatch accuracy: 84.0%
Validation accuracy: 85.8%
Minibatch loss at step 1000: 0.526706
Minibatch accuracy: 89.5%
Validation accuracy: 87.3%
Minibatch loss at step 1500: 0.508546
Minibatch accuracy: 90.2%
Validation accuracy: 88.2%
Minibatch loss at step 2000: 0.524380
Minibatch accuracy: 87.9%
Validation accuracy: 88.7%
Minibatch loss at step 2500: 0.629113
Minibatch accuracy: 85.2%
Validation accuracy: 89.1%
Minibatch loss at step 3000: 0.574620
Minibatch accuracy: 85.5%
Validation accuracy: 89.5%
Test accuracy: 94.8%


In the final convolutional network model, l2-regularization and learning rate decay were applied. Batch_size increased to 128 (a larger batch_size slows down the computation). The weights of the fully connected layer (F6) was initialized with a lower standard deviation of 0.01 (instead of 0.1). These adjustments result in a test accuracy of 94.8% for 3001 iterations. Dropout was not applied because it does not help improve the accuracy.  

In [95]:
# LeNet-5: c1->s2->c3->s4->c5->f6->output
batch_size = 128 #16
patch_size = 5
# C1 is a convolutional layer withfeature maps
# Each unit in each feature map is connected to a 5x5 
# neighborhood in the input.
c1_depth = 6 
# C3 is a convolutional layer with 16 feature maps
# Each unit in each feature map is connected to a 5x5 
# neighborhood in the input.
c3_depth = 16
# C5 is a convolutional layer with 120 feature maps
# Each unit in each feature map is connected to a 5x5 
# neighborhood in the input.
c5_depth = 120
# F6 contains 84 units
num_hidden = 84
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  c1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, c1_depth], stddev=0.1))
  c1_biases = tf.Variable(tf.zeros([c1_depth]))
  c3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, c1_depth, c3_depth], stddev=0.1))
  c3_biases = tf.Variable(tf.constant(1.0, shape=[c3_depth]))
  c5_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, c3_depth, c5_depth], stddev=0.1))
  c5_biases = tf.Variable(tf.constant(1.0, shape=[c5_depth]))
  c5_conv_dim = (((((image_size+1)//2)+1)//2)+1)//2
  f6_weights = tf.Variable(tf.truncated_normal(
    [c5_conv_dim * c5_conv_dim * c5_depth, num_hidden], stddev=0.01)) #
  f6_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  out_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  out_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, c1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + c1_biases)
    maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(maxpool, c3_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + c3_biases)
    maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(maxpool, c5_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + c5_biases)
    #maxpool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # dropout should be applied between 
    # the final convolutiona and the first fully connected lyaer 
    drop = tf.nn.dropout(hidden, 1.0)
    
    shape = drop.get_shape().as_list()
    reshape = tf.reshape(drop, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, f6_weights) + f6_biases)
    return tf.matmul(hidden, out_weights) + out_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  beta = 1e-3  #L-2 regularization which penalizes large weights
  R = tf.nn.l2_loss(c1_weights) + tf.nn.l2_loss(c3_weights) + tf.nn.l2_loss(c5_weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta * R
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 100, 0.96)
  
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run() #tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 2.985373
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.598700
Minibatch accuracy: 85.2%
Validation accuracy: 89.3%
Minibatch loss at step 4000: 0.572505
Minibatch accuracy: 85.9%
Validation accuracy: 90.3%
Minibatch loss at step 6000: 0.464664
Minibatch accuracy: 89.1%
Validation accuracy: 90.7%
Minibatch loss at step 8000: 0.404684
Minibatch accuracy: 93.8%
Validation accuracy: 90.7%
Minibatch loss at step 10000: 0.385944
Minibatch accuracy: 91.4%
Validation accuracy: 90.8%
Minibatch loss at step 12000: 0.466862
Minibatch accuracy: 92.2%
Validation accuracy: 90.8%
Minibatch loss at step 14000: 0.406365
Minibatch accuracy: 93.0%
Validation accuracy: 90.8%
Minibatch loss at step 16000: 0.380753
Minibatch accuracy: 91.4%
Validation accuracy: 90.8%
Minibatch loss at step 18000: 0.317793
Minibatch accuracy: 94.5%
Validation accuracy: 90.8%
Minibatch loss at step 20000: 0.405148
Minibatch accuracy: 89.1%
Validation 